In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import glob
import os
import time
import json
import cv2
from IPython.display import display

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import load_model


In [2]:
root = r'D:\桌面\基於深度學習之視覺辨識專論'
testPath = os.path.join(root, 'testing_data')
trainPath = os.path.join(root, 'training_data')
enocdePath = os.path.join(root, 'encodes.json')
modelPath = os.path.join(root, 'model_1_29-0.86.h5')
predictPath = os.path.join(root, f'predict_{round(time.time())}.csv')
imgSize = (224, 224)
preprocess = preprocess_input

print(f'Testing data path: {testPath}')
print(f'Training data path: {trainPath}')
print(f'Label encode path: {enocdePath}')
print(f'Loaded model path: {modelPath}')
print(f'Prediction data path: {predictPath}')
print(f'Size of input images: {imgSize}')
print(f'Preprocessing function: {preprocess}')

Testing data path: D:\桌面\基於深度學習之視覺辨識專論/testing_data
Training data path: D:\桌面\基於深度學習之視覺辨識專論/training_data
Label encode path: D:\桌面\基於深度學習之視覺辨識專論/encodes.json
Loaded model path: D:\桌面\基於深度學習之視覺辨識專論/model_1_29-0.86.h5
Prediction data path: D:\桌面\基於深度學習之視覺辨識專論/predict_1605186338.csv
Size of input images: (224, 224)
Preprocessing function: <function preprocess_input at 0x7fed0373db70>


In [ ]:
#-------------------------------------------------------------------------------
print('Reading testing set ...')
testImgPaths = pd.DataFrame(glob.glob(os.path.join(testPath, '*.jpg')), columns = ['path'])
testImgGen = ImageDataGenerator(preprocessing_function=preprocess)
testImgFlow = testImgGen.flow_from_dataframe(dataframe=testImgPaths, 
                        x_col='path',
                        target_size=imgSize, 
                        batch_size=1, 
                        interpolation='bilinear',
                        class_mode=None,
                        shuffle=False)

#-------------------------------------------------------------------------------
print('Reading encodes of labels ...')
with open(enocdePath, 'r', encoding='utf-8') as f:
    encodes = {}
    for key, value in json.load(f).items():
        encodes[int(value)] = key

#-------------------------------------------------------------------------------
print('Reading TF model ...')
classifier = load_model(modelPath)

#-------------------------------------------------------------------------------
print('Computing prediction ...')
imgPred = classifier.predict(testImgFlow, 1, 1, steps=testImgFlow.n)

#-------------------------------------------------------------------------------
print('Generating csv file of the prediction ...')
predict = pd.DataFrame()
predict['id'] = testImgPaths['path'].apply(lambda x: '%06d'%(int(os.path.split(x)[-1].replace('.jpg', ''))))
predict['label'] = np.argmax(imgPred, axis=1)
predict['label'] = predict['label'].map(encodes)
predict = predict.sort_values('id', ignore_index=True)
predict.to_csv(predictPath, index=False)
predict


Reading testing set ...
Found 5000 validated image filenames.
Reading encodes of labels ...
Reading TF model ...
Computing prediction ...
5000/5000 [==============================] - 2610s 522ms/step
Generating csv file of the prediction ...


,id,label
0,000004,AM General Hummer SUV 2000
1,000005,AM General Hummer SUV 2000
2,000006,HUMMER H2 SUT Crew Cab 2009
3,000008,AM General Hummer SUV 2000
4,000019,AM General Hummer SUV 2000
...,...,...
4995,016172,smart fortwo Convertible 2012
4996,016175,smart fortwo Convertible 2012
4997,016176,smart fortwo Convertible 2012
4998,016180,smart fortwo Convertible 2012
